<a href="https://colab.research.google.com/github/timeseriesAI/tsai/blob/master/tutorial_nbs/05_TS_archs_comparison.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

created by Ignacio Oguiza - email: timeseriesAI@gmail.com

# Import libraries 📚

In [ ]:
# # **************** UNCOMMENT AND RUN THIS CELL IF YOU NEED TO INSTALL/ UPGRADE TSAI ****************
# stable = True # Set to True for latest pip version or False for main branch in GitHub
# !pip install {"tsai -U" if stable else "git+https://github.com/timeseriesAI/tsai.git"} >> /dev/null

In [ ]:
from tsai.all import *
my_setup()

tsai       : 0.2.3
fastai     : 2.1.4
fastcore   : 1.3.2
torch      : 1.7.0+cu101


# Experiments 🧪

I've run a small test to show how you can build any model in the `tsai` library using the `create_model` functions.

The esiest way to do it is pass the architecture you want to use, a `TSDataLoaders` and any `kwargs` you'd like to use. The `create_model` function will pick up the necessary data from the `TSDataLoaders` object.

I've used 2 multivariate time series datasets. As you can see, it's difficult to predict which architecture will work best for which dataset.

Please, bear in mind that this is a very simple test without any hyperparameter tuning.

In [ ]:
dsid = 'NATOPS' 
bs = 64
X, y, splits = get_UCR_data(dsid, return_split=False)
print(X.shape)
tfms  = [None, [Categorize()]]
dsets = TSDatasets(X, y, tfms=tfms, splits=splits)
dls   = TSDataLoaders.from_dsets(dsets.train, dsets.valid, bs=[bs, bs*2])

archs = [(FCN, {}), (ResNet, {}), (xresnet1d34, {}), (ResCNN, {}), 
         (LSTM, {'n_layers':1, 'bidirectional': False}), (LSTM, {'n_layers':2, 'bidirectional': False}), (LSTM, {'n_layers':3, 'bidirectional': False}), 
         (LSTM, {'n_layers':1, 'bidirectional': True}), (LSTM, {'n_layers':2, 'bidirectional': True}), (LSTM, {'n_layers':3, 'bidirectional': True}),
         (LSTM_FCN, {}), (LSTM_FCN, {'shuffle': False}), (InceptionTime, {}), (XceptionTime, {}), (OmniScaleCNN, {}), (mWDN, {'levels': 4})]

results = pd.DataFrame(columns=['arch', 'hyperparams', 'total params', 'train loss', 'valid loss', 'accuracy', 'time'])
for i, (arch, k) in enumerate(archs):
    model = create_model(arch, dls=dls, **k)
    print(model.__class__.__name__)
    learn = Learner(dls, model,  metrics=accuracy)
    start = time.time()
    learn.fit_one_cycle(100, 1e-3)
    elapsed = time.time() - start
    vals = learn.recorder.values[-1]
    results.loc[i] = [arch.__name__, k, total_params(model)[0], vals[0], vals[1], vals[2], int(elapsed)]
    results.sort_values(by='accuracy', ascending=False, ignore_index=True, inplace=True)
    clear_output()
    display(results)

,arch,hyperparams,total params,train loss,valid loss,accuracy,time
0,OmniScaleCNN,{},5239596,0.028580,0.069324,0.983333,14
1,ResNet,{},490758,0.025631,0.094521,0.972222,7
2,LSTM_FCN,{},347246,0.071825,0.100571,0.972222,5
3,LSTM_FCN,{'shuffle': False},336446,0.076918,0.107438,0.966667,6
4,FCN,{},285446,0.070331,0.122366,0.955556,5
5,XceptionTime,{},403420,0.395302,0.491704,0.955556,10
6,ResCNN,{},268551,0.044637,0.148163,0.944444,6
7,InceptionTime,{},460038,0.029208,0.152726,0.933333,9
8,mWDN,{'levels': 4},467038,0.030618,0.199803,0.933333,10
9,xresnet1d34,{},7232518,0.025482,0.406264,0.888889,15


In [ ]:
dsid = 'LSST' 
bs = 64
X, y, splits = get_UCR_data(dsid, return_split=False)
print(X.shape)
tfms  = [None, [Categorize()]]
dsets = TSDatasets(X, y, tfms=tfms, splits=splits)
dls   = TSDataLoaders.from_dsets(dsets.train, dsets.valid, bs=[bs, bs*2])

archs = [(FCN, {}), (ResNet, {}), (xresnet1d34, {}), (ResCNN, {}), 
         (LSTM, {'n_layers':1, 'bidirectional': False}), (LSTM, {'n_layers':2, 'bidirectional': False}), (LSTM, {'n_layers':3, 'bidirectional': False}), 
         (LSTM, {'n_layers':1, 'bidirectional': True}), (LSTM, {'n_layers':2, 'bidirectional': True}), (LSTM, {'n_layers':3, 'bidirectional': True}),
         (LSTM_FCN, {}), (LSTM_FCN, {'shuffle': False}), (InceptionTime, {}), (XceptionTime, {}), (OmniScaleCNN, {}), (mWDN, {'levels': 4})]

results = pd.DataFrame(columns=['arch', 'hyperparams', 'total params', 'train loss', 'valid loss', 'accuracy', 'time'])
for i, (arch, k) in enumerate(archs):
    model = create_model(arch, dls=dls, **k)
    print(model.__class__.__name__)
    learn = Learner(dls, model,  metrics=accuracy)
    start = time.time()
    learn.fit_one_cycle(100, 1e-3)
    elapsed = time.time() - start
    vals = learn.recorder.values[-1]
    results.loc[i] = [arch.__name__, k, total_params(model)[0], vals[0], vals[1], vals[2], int(elapsed)]
    results.sort_values(by='accuracy', ascending=False, ignore_index=True, inplace=True)
    clear_output()
    display(results)

,arch,hyperparams,total params,train loss,valid loss,accuracy,time
0,XceptionTime,{},401580,0.074442,1.488087,0.678832,100
1,LSTM,"{'n_layers': 3, 'bidirectional': True}",572414,0.000234,2.650948,0.669505,98
2,ResCNN,{},260367,0.250915,1.266159,0.667883,60
3,LSTM_FCN,{'shuffle': False},314950,0.440930,1.236977,0.657340,59
4,ResNet,{},482574,0.030630,2.362593,0.650446,78
5,LSTM,"{'n_layers': 2, 'bidirectional': False}",125414,0.002532,2.354824,0.650041,56
6,LSTM,"{'n_layers': 2, 'bidirectional': True}",330814,0.000203,2.637241,0.644769,76
7,LSTM,"{'n_layers': 3, 'bidirectional': False}",206214,0.002217,2.461025,0.633820,66
8,LSTM,"{'n_layers': 1, 'bidirectional': False}",44614,0.054411,1.819723,0.616788,45
9,LSTM,"{'n_layers': 1, 'bidirectional': True}",89214,0.053924,1.760913,0.596918,56
